## Subscription analysis

In [1]:
"""
input: customer dataset from Shopify, customer report from ReCharge.
output: textfile of logit fit test result.
"""

import os
print(os.getcwd())

C:\Users\jaesu\PycharmProjects\Analysis\Subscriber analysis


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import glob
from pathlib import Path

# makes plots visible in pycharm
plt.style.use({'figure.facecolor':'white'})

# all_files = ['orders_export_1.csv', 'orders_export_2.csv', 'orders_export_3.csv']
# df_from_each_file = (pd.read_csv(f) for f in all_files)
# df = pd.concat(df_from_each_file, ignore_index=True)
# doesn't create a list, nor does it append to one

custRecharge = pd.read_csv('Customer_report_1-13.csv', delimiter=',')
custShopify = pd.read_csv('all_customers_shopify_1-13.csv', delimiter=',')

In [3]:
custRecharge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7083 entries, 0 to 7082
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   customer recharge id             7083 non-null   int64 
 1   customer first name              7079 non-null   object
 2   customer last name               7083 non-null   object
 3   customer email                   7083 non-null   object
 4   created at                       7083 non-null   object
 5   status                           7083 non-null   object
 6   shipping address 1               7083 non-null   object
 7   shipping address 2               1742 non-null   object
 8   shipping city                    7083 non-null   object
 9   shipping postal code             7042 non-null   object
 10  shipping province                5930 non-null   object
 11  shipping country                 7083 non-null   object
 12  shipping_phone                   7

In [ ]:
custShopify.info()

In [ ]:
custRecharge['status'] = custRecharge['status'].str.lower()

In [ ]:
custRecharge.rename(columns={'customer email' : 'Email'}, inplace=True)

In [ ]:
mergedDf = custRecharge.merge(custShopify, how='inner', on='Email')
# mergedDf.to_csv("MergedDf.csv")

In [ ]:
mergedDf.info()

## 1. Customer engagement (active vs inactive)

In [ ]:
mergedDf['Accepts Marketing']

In [ ]:
mergedDf['Accepts Marketing'] = mergedDf['Accepts Marketing'].apply(lambda x: 0 if x=='no' else 1)

In [ ]:
mergedDf['engaged'] = mergedDf['status'].apply(lambda x: 0 if x=='inactive' else 1)

In [ ]:
marketing_by_status_df = pd.DataFrame(mergedDf.groupby('engaged').count()['status'] / mergedDf.shape[0] * 100)

In [ ]:
marketing_by_status_df

In [ ]:
marketing_by_status_df.T

#### a. engagement by number of subscriptions

In [ ]:
mergedDf['Number Subscriptions'].value_counts()

In [ ]:
engagement_by_sub_type_df = pd.pivot_table(
    mergedDf,
    values='status',
    index='Number Subscriptions',
    columns='engaged',
    aggfunc=len
).fillna(0.0)

engagement_by_sub_type_df.columns = ['Not Engaged', 'Engaged']

In [ ]:
engagement_by_sub_type_df

#### b. engagement by country

In [ ]:
mergedDf['Country'].value_counts()

#### showing engagement by countries in numbers

In [ ]:
engagement_by_country_df = pd.pivot_table(
    mergedDf,
    values='status',
    index='Country',
    columns='engaged',
    aggfunc=len
).fillna(0.0)

pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)


engagement_by_country_df.columns = ['Not Engaged', 'Engaged']
engagement_by_country_df.sort_values(ascending=False, by=['Not Engaged', 'Engaged']).rename_axis(None, axis=1).reset_index()

#### showing engagement by countries on percentage

In [ ]:
func = lambda x: 100*x.count()/mergedDf.shape[0]

engagement_by_country_df = pd.pivot_table(
    mergedDf,
    values='status',
    index='Country',
    columns='engaged',
    aggfunc=func
).fillna(0.0)

In [ ]:
engagement_by_country_df.columns = ['Not Engaged', 'Engaged']
engagement_by_country_df = engagement_by_country_df.sort_values(ascending=False, by=['Not Engaged', 'Engaged'])

In [ ]:
countries_only = mergedDf['Country']
countries_count = countries_only.value_counts().rename_axis('Country').reset_index(name='Total')
countries_count

In [ ]:
new = pd.merge(engagement_by_country_df, countries_count, on='Country', how='left')
new

#### c. engagement by  marketing acceptance


In [ ]:
engagement_by_marketing = pd.pivot_table(
    mergedDf,
    values='status',
    index='Accepts Marketing',
    columns='engaged',
    aggfunc=len
).fillna(0.0)

engagement_by_marketing.columns = ['Not Engaged', 'Engaged']

##### Accepts Marketing = Subscribed to newletter - 0 means no, 1 means yes

In [ ]:
engagement_by_marketing

## Correlation

#### Feature Engineering - subscription duration

In [ ]:
pearsons = mergedDf.corr()

In [ ]:
pearsons

In [ ]:
kendall = mergedDf.corr(method='kendall')

In [ ]:
kendall

In [ ]:
spearmans = mergedDf.corr(method='spearman')

In [ ]:
spearmans

In [ ]:
from scipy.stats import kendalltau, pearsonr, spearmanr

In [ ]:
def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ['Past Orders Count', 'Number Active Subscriptions', 'Number Subscriptions', 'Accepts Marketing', 'Total Spent', 'Total Orders', 'engaged']
scatter_matrix(mergedDf[attributes], figsize=(18, 18))

## Regression Analysis

In [ ]:
import statsmodels.api as sm
import pandas.util.testing as tm

In [ ]:
mergedDf.describe()

In [ ]:
active = mergedDf[mergedDf['engaged']==1]
inactive = mergedDf[mergedDf['engaged']==0]

# active.drop(['customer recharge id', 'Shopify customer id'], axis=1)
del active['customer recharge id']
del active['Shopify customer id']

del inactive['Shopify customer id']
del inactive['customer recharge id']


In [ ]:
active.describe()

In [ ]:
inactive.describe()

In [ ]:
mergedDf.columns.to_list

In [ ]:
mergedDf.dtypes

In [ ]:
continuous_vars = [
                   'Past Orders Count',
                   'Number Active Subscription',
                   'Number Subscriptions',
                   'Total Spent',
                   'Total Orders'
]

In [ ]:
logit = sm.Logit(
    mergedDf['engaged'],
    mergedDf[['Past Orders Count',
              'Number Active Subscriptions',
              'Number Subscriptions',
              'Total Spent',
              'Total Orders']]
)

In [ ]:
logit_fit = logit.fit()

In [ ]:
logit_fit.summary()

#$$

#print the output of the result into a text file.
with open('logit_fit_test.txt', 'w') as f:
    f.write(logit_fit.summary().as_text())




